In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
sys.path.append('..')

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV

from get_data import get_data, get_splitted_data
from models.active_model import ActiveModel
from models.strategy import *
from models.utils import ObstructedY

from misc.config import c
data_dir = c["DATA_DIR"]

In [ ]:
seed = 666
strategy_args = {'batch_size': 10, 'seed': seed}
comp = [['5ht7', 'ExtFP']]
loader = ["get_splitted_data",
          {"n_folds": 2,
           "seed":666,
           "test_size":0.0}]
preprocess_fncs = []

sgd = SGDClassifier(random_state=seed, alpha=1)
model = ActiveModel(strategy=random_query, base_model=sgd)

folds, test_data, data_desc = get_data(comp, loader, preprocess_fncs).values()[0]
print data_desc
X = folds[0]['X_train']
y = ObstructedY(folds[0]['Y_train'])

X_test = folds[0]['X_valid']
y_test = folds[0]['Y_valid']
model.fit(X, y, strategy_args=strategy_args)
p = model.predict(X_test)
accuracy_score(p, y_test)

In [ ]:
seed = 666
strategy_args = {'batch_size': 10, 'method': 'simple', 'seed': seed}
comp = [['5ht7', 'ExtFP']]
loader = ["get_splitted_data",
          {"n_folds": 2,
           "seed":666,
           "test_size":0.0}]
preprocess_fncs = []

folds, test_data, data_desc = get_data(comp, loader, preprocess_fncs).values()[0]
print data_desc
X = folds[0]['X_train']
y = ObstructedY(folds[0]['Y_train'])

X_test = folds[0]['X_valid']
y_test = folds[0]['Y_valid']

sgd = SGDClassifier(random_state=seed, alpha=1)
model = ActiveModel(strategy=uncertainty_sampling, base_model=sgd)

model.fit(X, y, strategy_args=strategy_args)
p = model.predict(X_test)
accuracy_score(p, y_test)

In [ ]:
from sklearn.svm import SVC

X = folds[0]['X_train']
y = ObstructedY(folds[0]['Y_train'])

X_test = folds[0]['X_valid']
y_test = folds[0]['Y_valid']

svm = SVC(C=0.01, kernel='linear', probability=True)

seed = 666
strategy_args = {'batch_size': 10, 'seed': seed, 'method': 'entropy'}
comp = [['5ht7', 'ExtFP']]
loader = ["get_splitted_data",
          {"n_folds": 2,
           "seed":666,
           "test_size":0.0}]
preprocess_fncs = []

model = ActiveModel(strategy=uncertainty_sampling, base_model=svm)
model.fit(X, y, strategy_args=strategy_args)
p = model.predict(X_test)
accuracy_score(p, y_test)

In [ ]:
X = folds[0]['X_train']
y = ObstructedY(folds[0]['Y_train'])

X_test = folds[0]['X_valid']
y_test = folds[0]['Y_valid']

svm = SVC(C=0.01, kernel='linear')

seed = 666
strategy_args = {'batch_size': 10, 'seed': seed, 'method': 'entropy', 'n_bags': 5}
comp = [['5ht7', 'ExtFP']]
loader = ["get_splitted_data",
          {"n_folds": 2,
           "seed":666,
           "test_size":0.0}]
preprocess_fncs = []

model = ActiveModel(strategy=query_by_bagging, base_model=svm)
model.fit(X, y, strategy_args=strategy_args)
p = model.predict(X_test)
accuracy_score(p, y_test)

In [ ]:
X = folds[0]['X_train']
y = ObstructedY(folds[0]['Y_train'])

X_test = folds[0]['X_valid']
y_test = folds[0]['Y_valid']

svm = SVC(C=0.01, kernel='linear', probability=True)

seed = 666
strategy_args = {'batch_size': 10, 'seed': seed, 'method': 'KL', 'n_bags': 5}
comp = [['5ht7', 'ExtFP']]
loader = ["get_splitted_data",
          {"n_folds": 2,
           "seed":666,
           "test_size":0.0}]
preprocess_fncs = []

model = ActiveModel(strategy=query_by_bagging, base_model=svm)
model.fit(X, y, strategy_args=strategy_args)
p = model.predict(X_test)
accuracy_score(p, y_test)

In [ ]:
p = np.array([clf.predict_proba(X_test) for clf in bc.estimators_])

In [ ]:
print p.shape
(p / pc).shape

In [ ]:
a = np.arange(6).reshape(3,2)
b = 2 * np.ones(6).reshape(3,2)
a/b

In [ ]:
np.mean(np.sum(p * np.log(p / pc), axis=2), axis=0).shape

In [ ]:
a = np.arange(10)
a[-2:]